In [ ]:
"""
MCP Database Report Generator - Jupyter Notebook
Mountain Capital Partners - Ski Resort Data Analysis

This notebook generates comprehensive reports for all configured resorts.
It matches the functionality of main.py but in an interactive notebook format.
"""

import pandas as pd
from datetime import datetime
from report_generator import ReportGenerator
from config import RESORT_MAPPING

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ All modules imported successfully!")


In [ ]:
# ========== CONFIGURATION ==========

OUTPUT_DIR = "reports"  # Output directory for Excel reports

# Get list of resorts to process
# You can filter this list if you only want specific resorts
resorts = RESORT_MAPPING

print(f"📊 Configuration:")
print(f"   Output Directory: {OUTPUT_DIR}")
print(f"   Number of Resorts: {len(resorts)}")
print(f"\n🏔️  Resorts to process:")
for resort_config in resorts:
    print(f"   • {resort_config['resortName']} (Group: {resort_config['groupNum']})")

In [ ]:
# Initialize report generator
generator = ReportGenerator(OUTPUT_DIR)

saved_files = []

print(f"\n{'='*70}")
print(f"  Starting Batch Report Generation for {len(resorts)} Resorts")
print(f"{'='*70}\n")

# Generate comprehensive report for each resort
for i, resort_config in enumerate(resorts, 1):
    resort_name = resort_config.get('resortName')
    print(f"\n[{i}/{len(resorts)}] Processing {resort_name}...")
    
    try:
        file_path = generator.generate_comprehensive_report(
            resort_config=resort_config,
            run_date=datetime.now()
        )
        if file_path:
            saved_files.append(file_path)
            print(f"✅ Success!")
    except Exception as e:
        print(f"❌ Failed: {e}")
        import traceback
        traceback.print_exc()

print(f"\n{'='*70}")
print(f"  Generation Complete!")
print(f"{'='*70}")
print(f"\n✨ Successfully generated {len(saved_files)} out of {len(resorts)} reports")
print(f"\n📁 Saved files:")
for file_path in saved_files:
    print(f"   • {file_path}")


In [ ]:
# ============================================================
# OPTIONAL: Generate Report for a Single Resort
# ============================================================
# If you want to generate a report for just one resort instead
# of all resorts, modify and run the cell below


In [ ]:
# Example: Generate report for a single resort
# Uncomment and modify to run

single_resort_config = {
    "dbName": "Purgatory",
    "resortName": "PURGATORY",
    "groupNum": 46
}

generator = ReportGenerator(OUTPUT_DIR)
file_path = generator.generate_comprehensive_report(
    resort_config=single_resort_config,
    run_date=datetime.now()
)

print(f"\n✅ Report generated: {file_path}")


In [ ]:
# Display all available resort configurations
print("🏔️  Available Resort Configurations:\n")
for resort in RESORT_MAPPING:
    print(f"Resort: {resort['resortName']}")
    print(f"  Database: {resort['dbName']}")
    print(f"  Group Number: {resort['groupNum']}")
    print()
